# 1. 0I+3X+5Y+7Z

In [1]:
from math import *
import numpy as np
import scipy.linalg as la
import random 
import cmath 
import math

In [2]:
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute, Aer
import pennylane as qml
import numpy as np
from numpy import pi
simulator = Aer.get_backend('qasm_simulator')

from scipy.linalg import expm, sinm, cosm

In [34]:
def y_point_1(x):
    y_temp = []
    a1 =       24.26 
    a2 =      -1.085  
    b1 =       37.21  
    c1 =       7.283  
    c2 =     0.01759 
    d1 =        0.62  
    d2 =      0.7687  
    e =      0.8706  
    for i in x:
        temp = a1/(b1+c1*exp(-d1*i))+a2/(1+c2*exp(-d2*i))+e
        y_temp.append(temp)
    return(y_temp)

In [4]:
def theta_point_1(x):
    z_temp = []
    a1 =      -2.661  
    a2 =       4.206  
    c1 =    0.004604  
    c2 =    0.005635  
    d1 =       1.033  
    d2 =       1.307  
    e =      0.7446  
    for i in x:
        temp = a1/(1+c1*exp(-d1*i))+a2/(1+c2*exp(-d2*i))+e
        z_temp.append(temp)
    return(z_temp)

In [5]:
def measure_z_circuit(given_circuit):
    z_meas = QuantumCircuit(1,1)
    z_meas.append(given_circuit,[0])
    z_meas.measure(0,0)
    return z_meas

def measure_z(given_circuit, num_shots = 10000):

    z_meas = measure_z_circuit(given_circuit)
    
    result = execute(z_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(z_meas)

    if '0' not in counts:
        counts['0'] = 0
    if '1' not in counts:
        counts['1'] = 0

    total_counts = counts['0'] + counts['1']
    z = counts['0'] - counts['1']
    z = z / total_counts
    
    return z

def measure_x_circuit(given_circuit):
    
    x_meas = QuantumCircuit(1,1)
    x_meas.append(given_circuit,[0])
    x_meas.h(0)
    x_meas.measure(0,0)

    return x_meas

def measure_x(given_circuit, num_shots = 10000):
    
    x_meas = measure_x_circuit(given_circuit)
    
    result = execute(x_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(x_meas)

    if '0' not in counts:
        counts['0'] = 0
    if '1' not in counts:
        counts['1'] = 0

    total_counts = counts['0'] + counts['1'] 
    x = counts['0'] - counts['1']
    x = x / total_counts
    
    return x

def measure_y_circuit(given_circuit):
    
    y_meas = QuantumCircuit(1,1)
    y_meas.append(given_circuit,[0])
    y_meas.rz(-pi/2,0)
    y_meas.h(0)

    y_meas.measure(0,0)

    return y_meas

def measure_y(given_circuit, num_shots = 10000):
    
    y_meas = measure_y_circuit(given_circuit)
    
    result = execute(y_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(y_meas)

    if '0' not in counts:
        counts['0'] = 0
    if '1' not in counts:
        counts['1'] = 0

    total_counts = counts['0'] + counts['1'] 
    y = counts['0']  - counts['1']
    y = y / total_counts
    
    return y

In [6]:
X = np.matrix('0,1;1,0')
Y = np.matrix('0,-1j;1j,0')
Z = np.matrix('1,0;0,-1')
I = np.matrix('1,0;0,1')
e = 3
a=1+e
b = 2.3+e
c = 6.2+e
d = 8.8+e
M = a*I+b*X+c*Y+d*Z

In [7]:
def mat(n):
    M_temp = n*I+(3+n)*X+(n+5)*Y+(n+7)*Z
    return(M_temp)

In [8]:
coeffs, obs_list = qml.utils.decompose_hamiltonian(M)
H = qml.Hamiltonian(coeffs, obs_list)

In [9]:
def coef(H):
    coeff = [0,0,0,0]
    h = str(H)
    for i in range (len(h)):
        p = ""
        if h[i] =="(":
            j = i+1 
            while h[j]!=")" :
                p=p+h[j]
                j+=1
            
            if h[j+3] == "I":
                coeff[0] = float(p)
                
            if h[j+3] == "X":
                coeff[1] = float(p)
                
            if h[j+3] == "Y":
                coeff[2] = float(p)
                
            if h[j+3] == "Z":
                coeff[3] = float(p)
    
    return(coeff)

In [10]:
def eigen_value_check(qc,H):
    num_shots = 100000
    z = measure_z(qc, num_shots = num_shots)
    x = measure_x(qc, num_shots = num_shots)
    y = measure_y(qc, num_shots = num_shots)
    coeff =coef(H) 
    ev = coeff[0]+coeff[1]*x+coeff[2]*y+coeff[3]*z
    
    return ev

In [11]:
qc_a = QuantumCircuit(1)
qc_a.initialize([ 0.32296885+0.j,-0.33861069-((1-0.33861069**2-0.32296885**2)**0.5)*1.j],[0])

In [12]:
eigen_value_check(qc_a,H)

-11.726477999999998

In [13]:
results = la.eig(M)

In [14]:
results[0]

array([ 19.87356293-1.13964669e-16j, -11.87356293-7.74213751e-16j])

In [15]:
min(results[0].real)

-11.873562927080991

In [16]:
def Rand(start, end, num): 
    res = [] 

    for j in range(num): 
        res.append(random.randint(start, end)) 

    return res 



In [38]:
num = 500
start = -500
end = 500
x = Rand(start, end, num)

In [39]:
def state(a,b):
    temp = (1-a**2)**0.5
    return([a,temp*cos(b)+temp*sin(b)*1j])
    

In [40]:
y = y_point_1(x)
t = theta_point_1(x)

In [41]:
data_list_1 = [0]*6
error_list_1 = []
exact_1 = []
predicted_1 = []
for i in range(len(x)):
    M = mat(x[i])
    results = la.eig(M)
    eigenval = min(results[0].real)
    
    
    
    qc= QuantumCircuit(1)
    qc.initialize(state(y[i],t[i]),[0])
    coeffs, obs_list = qml.utils.decompose_hamiltonian(M)
    H = qml.Hamiltonian(coeffs, obs_list)
    
    eigenval_predict = eigen_value_check(qc,H)
    
    
    error = (eigenval - eigenval_predict)/(eigenval)
    
    if error <0.05:
        data_list_1[0]+=1
    if 0.1>error :
        data_list_1[1]+=1
    if 0.15>error:
        data_list_1[2]+=1
    if 0.2>error :
        data_list_1[3]+=1
    if 0.25>error:
        data_list_1[4]+=1
    error_list_1.append(error)
    data_list_1[5]+=1
    
    exact_1.append(eigenval)
    predicted_1.append(eigenval_predict)
    

In [42]:
print("Total Test Points", data_list_1[5])
print("Prediction With error less then 5% :- ", data_list_1[0])
print("Prediction With error less then 10% :- ", data_list_1[1])
print("Prediction With error less then 15% :- ", data_list_1[2])
print("Prediction With error less then 20% :- ", data_list_1[3])
print("Prediction With error less then 25% :- ", data_list_1[4])

Total Test Points 500
Prediction With error less then 5% :-  249
Prediction With error less then 10% :-  249
Prediction With error less then 15% :-  249
Prediction With error less then 20% :-  249
Prediction With error less then 25% :-  251


In [61]:
print(0,".   x","             exact","      --      ","  predicted","    --    ","error")
for i in range(len(x)):
    print(i+1,". ",x[i],"    ",exact_1[i],"  --  ",predicted_1[i],"  --  ",error_list_1[i]*100,"%")

0 .   x              exact       --         predicted     --     error
1 .  -33      -81.5798312059645   --   -81.49495999999999   --   0.10403454470288076 %
2 .  -403      -1092.3620239032616   --   -1089.5442   --   0.25795696313140165 %
3 .  141      -111.89523522597257   --   61.43863999999999   --   154.9072888366735 %
4 .  -310      -838.2830680610543   --   -837.23654   --   0.12484184649881591 %
5 .  361      -272.93690537781447   --   162.20175999999998   --   159.4282989233249 %
6 .  243      -186.5579122772621   --   108.01688000000001   --   157.899918948206 %
7 .  277      -211.44651703129176   --   123.10367999999997   --   158.21977194440237 %
8 .  -191      -513.1738661033822   --   -512.04192   --   0.22057750367868414 %
9 .  187      -145.56578296631784   --   80.52864   --   155.3211327270732 %
10 .  100      -81.88732776089708   --   41.95559999999999   --   151.2357664454581 %
11 .  245      -188.021939397994   --   108.84219999999999   --   157.88803176293652 %
12

145 .  212      -163.86566749305527   --   92.74279999999999   --   156.59684631860455 %
146 .  -290      -783.6425832522959   --   -781.3638   --   0.2907936986832051 %
147 .  -206      -554.1537016893542   --   -553.2631   --   0.16071383925419222 %
148 .  -36      -89.7680202350802   --   -89.69376   --   0.082724599345885 %
149 .  -208      -559.6176901124288   --   -557.8086999999999   --   0.32325463336683585 %
150 .  36      -35.070387644925646   --   13.483440000000009   --   138.4467948758215 %
151 .  198      -153.61769011242876   --   86.91163999999998   --   156.57658303310745 %
152 .  82      -68.71496276083539   --   34.46056000000001   --   150.15000898703983 %
153 .  -276      -745.3942905490007   --   -744.23378   --   0.1556908288291255 %
154 .  -467      -1267.2124717848383   --   -1265.86188   --   0.10657974214348344 %
155 .  365      -275.86504039462164   --   159.55036000000004   --   157.83638251942514 %
156 .  -260      -701.6820123120249   --   -698.57598   --

284 .  357      -270.0087718684644   --   159.49499999999995   --   159.07030312248466 %
285 .  297      -226.0869908533379   --   133.31400000000005   --   158.96579873827437 %
286 .  363      -274.40097270085806   --   162.78900000000002   --   159.32522702004655 %
287 .  83      -69.44671200127603   --   35.10155999999999   --   150.54459597648776 %
288 .  462      -346.8726723038676   --   210.34341999999992   --   160.63995142740296 %
289 .  -144      -384.7716760750732   --   -384.63664   --   0.03509511834411294 %
290 .  -3      -7.47213595499958   --   -5.6290000000000004   --   24.66678826643061 %
291 .  -160      -428.482774121544   --   -428.43534   --   0.011070251690112005 %
292 .  -177      -474.926165349739   --   -474.90616   --   0.004212307343450408 %
293 .  -121      -321.9378013217024   --   -321.45   --   0.15152036191455992 %
294 .  171      -133.85406344675806   --   75.03299999999999   --   156.05582532789165 %
295 .  267      -204.12631002736416   --   117.0799

433 .  -174      -466.73025125531524   --   -466.32993999999997   --   0.08576929698441405 %
434 .  340      -257.56422248993454   --   153.49762000000004   --   159.59586254492262 %
435 .  54      -48.23013254417702   --   21.61876   --   144.8241770436729 %
436 .  325      -246.58376463996944   --   147.08072   --   159.6473657601703 %
437 .  -68      -177.1558518816101   --   -177.34602   --   -0.10734509550212643 %
438 .  -108      -286.42365314049596   --   -286.13252   --   0.1016442382826366 %
439 .  -477      -1294.5328739567599   --   -1291.75544   --   0.21455105641857578 %
440 .  398      -300.02220595049846   --   178.53773999999996   --   159.50817521468977 %
441 .  403      -303.68238976219016   --   183.73399999999998   --   160.50202652313155 %
442 .  -398      -1078.7018436878218   --   -1077.17276   --   0.14175220861719287 %
443 .  -144      -384.7716760750732   --   -385.09448   --   -0.0838949291225427 %
444 .  23      -25.57983120596448   --   7.38804   --   128.8

# 1. 0I+5X+30Y+20Z

In [45]:
def mat_2(n):
    M_temp = n*I+(5+n)*X+(n+30)*Y+(n+20)*Z
    return(M_temp)

In [46]:
def y_point_2(x):
    y_temp = []
    a1 =     0.00185  
    b1 =   -0.004296  
    c1 =  -0.0003929  
    d1 =      0.1289 
    e =      0.8807 
    for i in x:
        temp = a1/(b1+c1*exp(-d1*i))+e
        y_temp.append(temp)
    return(y_temp)

In [47]:
def theta_point_2(x):
    z_temp = []
    a =      27.72 
    a2 =       -26.19
    b =    0.1275
    b2 =    0.1151
    c =       0.09268
    c2 =       0.09086
    e =      0.6415
    for i in x:
        temp = a/(1+b*exp(-c*i))+a2/(1+b2*exp(-c2*i))+d
        z_temp.append(temp)
    return(z_temp)

In [48]:
y2 = y_point_2(x)
t2 = theta_point_2(x)

In [50]:
data_list_2 = [0]*6
error_list_2 = []
exact_2 = []
predicted_2 = []
for i in range(len(x)):
    M = mat_2(x[i])
    results = la.eig(M)
    eigenval = min(results[0].real)
    
    
    
    qc= QuantumCircuit(1)
    qc.initialize(state(y[i],t[i]),[0])
    coeffs, obs_list = qml.utils.decompose_hamiltonian(M)
    H = qml.Hamiltonian(coeffs, obs_list)
    
    eigenval_predict = eigen_value_check(qc,H)
    
    
    error = (eigenval - eigenval_predict)/(eigenval)
    
    if error <0.05:
        data_list_2[0]+=1
    if 0.1>error :
        data_list_2[1]+=1
    if 0.15>error:
        data_list_2[2]+=1
    if 0.2>error :
        data_list_2[3]+=1
    if 0.25>error:
        data_list_2[4]+=1
    error_list_2.append(error)
    data_list_2[5]+=1
    
    exact_2.append(eigenval)
    predicted_2.append(eigenval_predict)
    

In [51]:
print("Total Test Points", data_list_1[5])
print("Prediction With error less then 5% :- ", data_list_2[0])
print("Prediction With error less then 10% :- ", data_list_2[1])
print("Prediction With error less then 15% :- ", data_list_2[2])
print("Prediction With error less then 20% :- ", data_list_2[3])
print("Prediction With error less then 25% :- ", data_list_2[4])

Total Test Points 500
Prediction With error less then 5% :-  223
Prediction With error less then 10% :-  232
Prediction With error less then 15% :-  235
Prediction With error less then 20% :-  236
Prediction With error less then 25% :-  236


In [59]:
print(0,".   x","             exact","      --      ","  predicted","    --    ","error")
for i in range(len(x)):
    print(i+1,". ",x[i],"    ",exact_2[i],"  --  ",predicted_2[i],"  --  ",error_list_2[i]*100,"%")

0 .   x              exact       --         predicted     --     error
1 .  -33      -64.01612483854166   --   -59.21364   --   7.501992428711128 %
2 .  -403      -1069.4998124530869   --   -1066.72238   --   0.2596945245568953 %
3 .  141      -135.54656027511896   --   68.79598   --   150.7545007858294 %
4 .  -310      -815.4948070949888   --   -814.5880000000001   --   0.111197163623765 %
5 .  361      -296.26554755288976   --   165.90015999999997   --   155.99711521312926 %
6 .  243      -209.99227366479442   --   113.98542   --   154.28076853053753 %
7 .  277      -234.84177242581524   --   129.04140000000004   --   154.9482311715916 %
8 .  -191      -490.59639517190476   --   -489.1983   --   0.2849786883197237 %
9 .  187      -169.09268456400508   --   87.69082000000002   --   151.85961783391477 %
10 .  100      -105.73040611440982   --   48.14239999999998   --   145.53316474345667 %
11 .  245      -211.45372164108812   --   114.54080000000002   --   154.1682591874246 %
12 .  495

167 .  15      -45.41522986797286   --   9.7271   --   121.41814547295647 %
168 .  -325      -856.4602525118878   --   -855.1696000000001   --   0.15069613658105263 %
169 .  197      -176.3925548266864   --   92.99032000000003   --   152.71782592602457 %
170 .  166      -153.77023000898626   --   77.51756   --   150.41129222182337 %
171 .  -186      -476.95188605678425   --   -476.15644   --   0.16677700204954604 %
172 .  290      -244.34539391670626   --   136.0554   --   155.68158982623558 %
173 .  -35      -68.91164991562636   --   -64.5154   --   6.379545288799515 %
174 .  0      -36.40054944640259   --   -2.3024000000000004   --   93.67482075129078 %
175 .  -339      -894.6959600357013   --   -894.9870199999999   --   -0.032531717734254985 %
176 .  -11      -32.86321110907544   --   0.6529199999999991   --   101.98678089561277 %
177 .  -176      -449.66585464759737   --   -449.2578   --   0.0907461937302729 %
178 .  321      -267.0119046413941   --   151.11566000000002   --   156.

319 .  -50      -107.66281297335394   --   -105.3177   --   2.1782014686299713 %
320 .  -119      -294.26551286548073   --   -293.73082   --   0.18170422360202396 %
321 .  -323      -850.9981060572097   --   -848.7082800000001   --   0.26907534116835374 %
322 .  -346      -913.8142301844855   --   -913.05288   --   0.08331564111579123 %
323 .  -188      -482.4095786485216   --   -481.12618   --   0.2660392134246297 %
324 .  -271      -708.9931506313769   --   -708.54918   --   0.06261987594401647 %
325 .  233      -202.68566650740303   --   110.84278   --   154.68703431771854 %
326 .  352      -289.68294351650025   --   162.56889999999999   --   156.1195968345786 %
327 .  -299      -785.4545199707777   --   -784.73088   --   0.09213009186129277 %
328 .  249      -214.3767365761902   --   116.59302   --   154.38697400758429 %
329 .  453      -363.56720482762466   --   210.83685999999994   --   157.99116564981773 %
330 .  -136      -340.5800576791395   --   -339.01742   --   0.4588165524

472 .  -99      -239.8474351914155   --   -238.53870000000003   --   0.5456531942361671 %
473 .  -191      -490.59639517190476   --   -489.46308   --   0.23100764356567602 %
474 .  -219      -567.0201143612248   --   -566.10478   --   0.16142890490860348 %
475 .  -86      -204.5453499720677   --   -204.19513999999998   --   0.1712138516546773 %
476 .  -314      -826.4187740510688   --   -824.6593799999999   --   0.21289376600732093 %
477 .  287      -242.15215203190854   --   130.85108   --   154.03671984825377 %
478 .  -103      -250.72271321634997   --   -250.08326   --   0.2550439918852465 %
479 .  -3      -34.96873472629156   --   -5.618780000000001   --   83.93198940716756 %
480 .  -271      -708.9931506313769   --   -709.9900399999999   --   -0.1406063468645969 %
481 .  -440      -1170.5648499620006   --   -1166.8265999999999   --   0.31935436657969773 %
482 .  -47      -99.74466797696235   --   -97.35213999999999   --   2.398652504928834 %
483 .  -19      -36.832554500127   --  